In [16]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing import sequence
from string import punctuation

import numpy as np
import pandas as pd
import torch
import os

In [17]:
#== run this if using Kaushik's computer
# with open('C:/Users/kaush/Desktop/aclImdb/train/trainReviews.txt', 'r', encoding="utf8") as f:
#     reviews = f.read()
# labels = pd.read_csv('C:/Users/kaush/Desktop/aclImdb/train/IMDBValuesTrain.txt', sep='delimiter', names=['Score'], engine = 'python')
 
#== run this if using Zoe's computer
with open('C:/Users/huang/Documents/pythonUTIR_NLP/UTIR-Sentiment-Analysis/aclImdb/train/trainReviews.txt', 'r', encoding="utf8") as f:
    reviews = f.read()
labels = pd.read_csv('C:/Users/huang/Documents/pythonUTIR_NLP/UTIR-Sentiment-Analysis/aclImdb/train/IMDBValuesTrain.txt', sep='delimiter', names=['Score'], engine = 'python')
    
    


In [18]:
reviews = reviews.lower()

In [19]:
all_text = ''.join([c for c in reviews if c not in punctuation])

reviews_split = all_text.split('\n')

labels = labels.to_numpy()


In [20]:
all_text2 = ' '.join(reviews_split)

words = all_text2.split()

count_words = Counter(words)

total_words = len(words)
sorted_words = count_words.most_common(total_words)

In [21]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}



In [22]:
# Change Review Score to Binary Score
encoded_labels = [0 if label <= 5 else 1 for label in labels]
encoded_labels = np.array(encoded_labels)
print(encoded_labels)

[0 0 0 ... 0 0 0]


In [27]:
reviews_len = [len(x) for x in reviews_int]
#reviews_int = [reviews_int[i] for i, l in enumerate(reviews_len) if l > 0]   #do we need this? 
encoded_labels = [encoded_labels[i] for i, l in enumerate(reviews_len) if l > 0]

In [28]:
reviews_int = []
for reviews in reviews_split:
    r = [vocab_to_int[w] for w in reviews.split()]
    reviews_int.append(r)


In [29]:
seq_length = 200
def pad_features(reviews_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(reviews_int), seq_length), dtype = int)
    
    for i, review in enumerate(reviews_int):
        review_len = len(review)
        
        if review_len <= seq_length:
            zeroes = list(np.zeros(seq_length-review_len))
            new = zeroes+review
        elif review_len > seq_length:
            new = review[0:seq_length]
        
        features[i,:] = np.array(new)
    
    return features

pad_features(reviews_int, seq_length)

array([[    0,     0,     0, ...,    25,   115,  3481],
       [    0,     0,     0, ...,    55,   204,  1888],
       [    0,     0,     0, ...,    58,   877,   110],
       ...,
       [   45,   104,    11, ...,     2,  4940, 21477],
       [    0,     0,     0, ...,     8,     6,   904],
       [    0,     0,     0, ...,     0,     0,     0]])

In [ ]:
count_words = Counter(df_reviews_train['Review'])
total_words = len(df_reviews_train['Review'])
sorted_words = count_words.most_common(total_words)
print(count_words)